# Clustering Crypto

In [73]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [74]:
# Load the crypto_data.csv dataset.
file = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [75]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]

In [76]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [77]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns = 'IsTrading')

In [78]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [79]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]

In [80]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = crypto_df
crypto_names_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [81]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns = 'CoinName')


In [82]:
crypto_df.dtypes

Unnamed: 0          object
Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [83]:
# Use get_dummies() to create variables for text features.
crypto_encoded = pd.get_dummies(data=crypto_df, columns= ['Unnamed: 0','Algorithm', 'ProofType'])

In [84]:
# Standardize the data with StandardScaler().
crypto_df_scaled = StandardScaler().fit_transform(crypto_encoded)
crypto_df_scaled

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , 23.06512519, ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [85]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_df_scaled)

In [86]:
# Create a DataFrame with the three principal components.
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index
)
pca_df.head()

,PC 1,PC 2,PC 3
0,-0.618770,0.702504,-0.434460
2,-0.599893,0.701081,-0.434929
5,2.441984,1.544263,-0.418459
7,-0.076820,-1.344643,0.227879
8,0.086548,-1.860537,0.431680


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [87]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Visualization", xlabel="Number of Clusters", ylabel="Inertia")


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [88]:
# Initialize the K-Means model.
model = KMeans(n_clusters=6, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)

In [89]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.DataFrame({
    "Algorithm": crypto_df.Algorithm,
    "ProofType": crypto_df.ProofType,
    "TotalCoinsMined": crypto_df.TotalCoinsMined,
    "TotalCoinSupply": crypto_df.TotalCoinSupply,
    "PC 1": pca_df['PC 1'],
    "PC 2": pca_df['PC 2'],
    "PC 3": pca_df['PC 3']
    },
)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = crypto_names_df.CoinName

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = predictions
clustered_df.head()

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.618770,0.702504,-0.434460,42 Coin,0
2,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.599893,0.701081,-0.434929,404Coin,0
5,X13,PoW/PoS,2.927942e+10,314159265359,2.441984,1.544263,-0.418459,EliteCoin,5
7,SHA-256,PoW,1.792718e+07,21000000,-0.076820,-1.344643,0.227879,Bitcoin,1
8,Ethash,PoW,1.076842e+08,0,0.086548,-1.860537,0.431680,Ethereum,1
9,Scrypt,PoW,6.303924e+07,84000000,-0.159911,-1.342798,0.076694,Litecoin,1
10,X11,PoW/PoS,9.031294e+06,22000000,-0.550310,1.272485,-0.338636,Dash,0
11,CryptoNight-V7,PoW,1.720114e+07,0,0.045404,-2.326663,0.596482,Monero,1
12,Ethash,PoW,1.133597e+08,210000000,0.089165,-1.863328,0.430500,Ethereum Classic,1
13,Equihash,PoW,7.383056e+06,21000000,0.063270,-2.240916,0.333930,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [90]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name='CoinName',
    hover_data= ['Algorithm'],
    color="Class",
    symbol="Class",
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [91]:
# Create a table with tradable cryptocurrencies.
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
clustered_df.hvplot.table(columns)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [92]:
# Print the total number of tradable cryptocurrencies.
count= clustered_df['CoinName'].count()
print(f'There are {count} tradable cryptocurrencies')

There are 533 tradable cryptocurrencies


In [93]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_cluster_data = scaler.fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
print(scaled_cluster_data)

[[4.20000000e-11 5.94230127e-03]
 [5.32000000e-04 7.00182308e-03]
 [3.14159265e-01 3.53420682e-02]
 ...
 [1.40022261e-03 6.92655266e-03]
 [2.10000000e-05 5.94962775e-03]
 [1.00000000e-06 5.94243008e-03]]


In [94]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
new_clustered_df = pd.DataFrame(scaled_cluster_data, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE
new_clustered_df = pd.DataFrame({
    "TotalCoinSupply": new_clustered_df["TotalCoinSupply"],
    "TotalCoinsMined": new_clustered_df["TotalCoinsMined"],
    "CoinName": clustered_df["CoinName"],
    "Class": clustered_df["Class"]
})

new_clustered_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
0,4.200000e-11,0.005942,42 Coin,0
2,5.320000e-04,0.007002,404Coin,0
5,3.141593e-01,0.035342,EliteCoin,5
7,2.100000e-05,0.005960,Bitcoin,1
8,0.000000e+00,0.006050,Ethereum,1
9,8.400000e-05,0.006006,Litecoin,1
10,2.200000e-05,0.005951,Dash,0
11,0.000000e+00,0.005960,Monero,1
12,2.100000e-04,0.006056,Ethereum Classic,1
13,2.100000e-05,0.005950,ZCash,1


In [95]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
new_clustered_df.hvplot(
    kind="scatter", 
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    c='Class', 
    colormap="Plasma_r", 
    hover_cols=['CoinName']
)


:Scatter   [TotalCoinsMined]   (TotalCoinSupply,Class,CoinName)